# Geospatial Map of Germany

This Jupyter Notebook contains code to demonstrate a potential visualization (a geospatial choropleth map) of standardized inpatient treatment costs of a selected ICD-10-diagnosis across german federal lands ("Bundesländer").

A similar graph at other administrative levels (e.g. german distrcits) can be easily made by changing a few lines of the code.

I identified the following necessary tasks:

1. find and load an adequate GeoFile (preferably a ShapeFile, shp.) --> I used an official ShapeFile from the Federal Ministry of Cartography of Germany

2. create random, but reproducable data to visualize standardized inpatient treatment costs (= e.g. costs per inhabitants)
--> data on inpatient treatment costs in Germany can be requested from "DeStatis", the Federal Ministry of Statistics in Germany ("DRG-Statistik"). However, data from the Ministry are not free. Therefore, I decided to create my own, random data.

3. create a plesant and informative graph with Matplotlib (other good options would have been Plotly or Seaborn)


## Setup:

In [ ]:
# import necessary modules
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

## __1.) find and load an adequate GeoFile__

At first, an adequate GeoFile had to be found and downloaded. I decided to look for officially approved GeoFiles and found the ones provided by the german Federal Ministry of Cartography. The GeoFiles for different administrative levels and in different scales can be found under the following link:

https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete.html

I used the data with the 1:2 500 000 scale which can be found under this link:

https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete/verwaltungsgebiete-1-2-500-000-stand-31-12-vg2500-12-31.html

Furthermore, I chose the GeoFile with the coordinate reference system (CRS) "3-degree Gauss-Kruger zone 3", which corresponds to the CRS EPSG:31467.

(for an introduction to coordinate reference systems and the use of geospatial data in python, for example enroll in the corresponding course on https://kaggle.com/learn/geospatial-analysis)

Last, but not least, I decided to use the file for the administrative level of the german federal lands ("Bundesländer"), which is marked by the abbreviation "LAN". 

In [ ]:
# read in GeoFile
inpatient_costs_germany = gpd.read_file("Path-to-GeoFile.shp")

# check CRS
print(inpatient_costs_germany.crs)

# check the data
print(inpatient_costs_germany.head())
print(inpatient_costs_germany.tail())
print(inpatient_costs_germany.columns.value)
inpatient_costs_germany.plot()
plt.show()

After checking the data, I noticed that the graph showed more boundaries than expected (there are only 16 federal lands in Germany). Therefore, I read the accompanying use file provided by the Ministry of Cartography ("Produktdokumentation VG2500"). This use file stated that there are different boundaries on the administrative level of the federal lands, depending on whether one wants to visualize the additional areas that belong to the German State but are located in water (concerns areas in the North Sea, Baltic Sea and Lake Constance). I was only interested in the boundaries located on the land mass of Germany. That's why I used the variable "GF" (which stands for "Geofactor") to restrict the data to the land mass of Germany (= value 9):

In [ ]:
# restrict the data
inpatient_costs_germany = inpatient_costs_germany.loc[inpatient_costs_germany['GF'] == 9]

# check the data again
inpatient_costs_germany.plot()
plt.show()
# this looks good now!

Then I restricted the DataFrame to the two necessary columns for the geospatial choropleth map: "GEN" (contains the names of the 16 german federal lands) & "geometry" (contains the polygons / geospatial data)

In [ ]:
inpatient_costs_germany = inpatient_costs_germany[['GEN', 'geometry']]
print(inpatient_costs_germany.shape)
print(inpatient_costs_germany.head())

## __2.) create random, but reproducable data__

With real data, one would have to extract, clean, format, and normalize the data of the "DRG-Statistik" before creating the geospatial choropleth map. In my case (with random data) none of these steps were necessary. I just had to create a random array of plausible data and merge it to the DataFrame. 

I chose a random ICD-10-diagnosis for which I simulated to provide inpatient treatment costs per inhabitant (= normalization to account for the varying number of inhabitants in the federal lands of Germany): ICD-C34 / "malignant neoplasm of bronchia and lung".

First, I had to come up with plausible data. Therefore, I looked for a study that calculated the average economic burden of lung cancer in the European Union. I found corresponding calculations in the following paper of Luengo-Fernandez et al. (2013): _"Economic burden of cancer across the European Union: a population-based cost analysis"_

The paper can be found in "The Lancet - Oncology" under the following link:
https://doi.org/10.1016/S1470-2045(13)70442-X

Luengo-Fernandez et al. (2013) calculated that all cancer disease in the European Union cost €126 billion in 2009.
Lung cancer had the highest economic burden of all cancer diseases, with costs of €18.8 billion - which are 15% of the total of €126 billion cancer costs in the EU. 
They also calculated that, on average, Germany spent €182 per inhabitant for health care services in relation to cancer diseases. 

In recourse to these calculations, I extrapolated the average cost for health care services in relation to lung cancer diseases in the following way:

total health care costs per inhabitant for all cancer diseases in Germany * proportion of costs of lung cancer diseases in relation to the total costs of all cancer diseases in the EU
--> €182 * 0.15 = €27.3 per inhabitant. 
So, I assumed that the average health care costs of lung cancer diseases in Germany per inhabitant were €27.3

In [ ]:
# set seed to ensure reproducability
np.random.seed(1)

# create random array with plausible data for all 16 federal lands of Germany
random_array = np.random.randint(low = 16, high = 53, size = 16)
print(random_array)

# check the random array (there should be a certain variability in the data, so that the choropleth graph looks nice; and the mean should approximate €27.3)
print(np.amin(random_array))
print(np.amax(random_array))
print(np.mean(random_array))
print(np.median(random_array))
print(np.std(random_array))
plt.hist(random_array)
plt.show()
# looks good to me! The mean is 27.5625, which approximates the €27.3 very well in my eyes.

# merge the array containing the random data with the DataFrame containing the geospatial data & check the resulting DataFrame
inpatient_costs_germany['values'] = random_array
print(inpatient_costs_germany.head())
print(inpatient_costs_germany.tail())
# perfect!

## __3.) create a pleasant and informative graph with Matplotlib__

I had already enrolled in the Kaggle course on geospatial mapping when I started this project, and therefore had an idea of how to use geospatial data and how to create a choropleth graph in general. 

However, I did not exactly know how to change specific aspects of the graph in order to make it even more appealing and informative (e.g. how to add the names of the federal lands appropriately). 
Therefore, I looked up tutorials on how to properly design choropleth graphs with Maptlotlib. 

For the specific graph I created in this Jupyter Notebook, I drew heavily on M. Rake Linggar A's directions and considerations in the following post on "Towards Data Science":
https://towardsdatascience.com/a-beginners-guide-to-create-a-choropleth-map-in-python-using-geopandas-and-matplotlib-9cc4175ab630

This was the code for creating the graph:

In [ ]:
# create figure and fitting axes for Matplotlib
fig, ax = plt.subplots(1, figsize ) (12.5, 10))

I always wondered if there was an easy way to remove the weird box around geospatial graphs that Matplotlib creates by default. Therefore, I was very thankful for this line of code provided by M. Rake Linggar A.:

In [ ]:
ax.axis('off')

The following code is not intuitive, at least it wasn't for me. It is not as easy as it ought to be to define a title and a subtitle with different fontsizes in Matplotlib. A quick fix was use 'plt.suptitle' to define the title of the graph, and then 'ax.set_title' to define the subtitle of the graph:

In [ ]:
plt.suptitle('Stationäre Behandlungskosten bösartiger Neubildungen der Bronchien der Lunge', fontsize = 25)
ax.set_title('Vergleich der standardisierten Kosten in den deutschen Bundesländern',
              fontdict = {'fontsize': '18', 'fontweight': '3'})

Then, I created the colorbar legend, added it to the figure, and created the map:

In [ ]:
# create colorbar legend and add it to the figure
sm = plt.cm.ScalarMappable(cmap = 'Reds', norm = plt.Normalize(vmin = 10, vmax = 55))
fig.colorbar(sm, orientation = 'horizontal', fraction = 0.036, pad = 0.05, aspect = 20, label = '€ / Einwohner:in')

# create map
inpatient_costs_germany.plot(column = 'value', cmap = 'Reds', linewidth = 0.8, ax = ax, edgecolor = '0.8')

Next, I had to create a column with representative coordinates of the 16 federal lands of Germany. These were the points where I planned to place the annotations on the geospatial choropleth map:

In [ ]:
# add column with representative coordinates of the 16 federal lands of Germany to the DataFrame
inpatient_costs_germany['coords'] = inpatient_costs_germany['geometry'].apply(lambda x: x.representative_point().coords[:])
inpatient_costs_germany['coords'] = [coords[0] for coords in inpatient_costs_germany['coords']]

In the last step, I had to place the annotations on the extracted representative coordinates. This code is more complex than I thought it would be, because there is a problem with the annotations of Berlin and Brandenburg: they are so near to one another that they overlap. 

I found directions on how to handle this problem in another post on "Towards Data Science":
https://towardsdatascience.com/how-to-realize-geo-visualization-with-python-during-corona-pandemic-f822d3cbf4fe

However, I wasn't very satisfied with the result when applying this approach. Therefore, I just used part of the code of this blog post and decided to add the coordinates for the annotation for "Brandenburg" manually:

In [ ]:
for idx, row in inpatient_costs_germany.iterrows():
    if idx == 11: # the index of "Brandenburg" is 11; so the code inside this If-Statement only applies to the federal land of "Brandenburg"
        plt.annotate(text = inpatient_costs_germany.loc[11, 'GEN'], xy = (3738000, 5788000), fontsize = 11)
    elif idx != 11:
        plt.annotate(text = row['GEN'], xy = row['coords'], verticalalignment = 'center', horizontalalignment = 'center', fontsize = 11)

And now, the choropleth map is ready and can be saved:

In [ ]:
plt.savefig('inpatient-treatment_costs_germany.png', dpi = 300)

You can find the graph in the "issues" section of my "PortfolioProjects" repository or under the following link:

https://user-images.githubusercontent.com/116423681/210179686-11b0ac1d-e1c6-4acb-9d9b-fbddde6f3cad.png